In [1]:
### Library Import Initialization

import numpy as np
import math
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import warnings
warnings.filterwarnings("ignore")

In [2]:
### Function to Import Stock Tickers and Calculate Final Stock Price

def import_stock_data(tickers, start_date):
    data = pd.DataFrame()
    if len([tickers]) == 1:
        data[tickers] = yf.download(tickers, start_date)['Adj Close']
        data = pd.DataFrame(data)
    else:
        for t in tickers:
            data[t] = yf.download(tickers, start_date)['Adj Close']
    return data

tickers = 'GOOG'
stock_data = import_stock_data(tickers, '2018-01-01')
# Get the Current Stock Price (Starting Node of Tree)
S_0 = stock_data[tickers].iloc[-1]
S_0

[*********************100%%**********************]  1 of 1 completed


135.24000549316406

In [3]:
### Sigma Calculation 

def compute_sigma(data):
    # Compute the standard deviation of returns
    sigma = np.std(data) / 100
    return sigma

sigma = compute_sigma(stock_data)
sigma

GOOG    0.326099
dtype: float64

In [4]:
### Function to Compute the Trinomial Model Option Price

def trinomial_model(S_0, K, T, r, sigma, option_type = 'call', n = 1000):
    """
    Parameters:
    S_0 (float): Current stock price (starting node of tree)
    K (float): Strike price of the option
    T (float): Time to expiration (in years)
    r (float): Risk-free interest rate
    sigma (float): Volatility of the underlying stock
    option_type (str): Type of option ('call' or 'put')
    n (int): Number of steps in the binomial tree
    Returns:
    float: Option price
    """
    # Compute Delta T
    dt = T / n
    # Compute up and down parameters
    u = np.exp(sigma * np.sqrt(3 * dt))
    d = 1 / u

    # Compute probability of Increase, Decrease, and No Change
    p_u = np.sqrt(dt / (12 * sigma**2)) * (r - (1/2 * sigma**2) + 1/6)
    p_d = -np.sqrt(dt / (12 * sigma**2)) * (r - (1/2 * sigma**2) + 1/6)
    p_m = 1 - p_u - p_d

    # Init arrays for strike prices and option values
    S_n = np.zeros(n + 1)
    option_values = np.zeros(n + 1)

    # Generate the stock prices at each node S_n (points on Trinomial Tree)
    ''' 
    The expression essentially computes the stock price at node n by starting from the initial price S_0​ and multiplying it by the 
    factors u and d corresponding to the number of up and down movements, respectively, needed to reach node n.
    S_n​ = S_0 * u^m * d^(n−m)
    m (int): Number of up movements from the initial node to node n, calculated as m = n - i
    '''
    